Import important libraries

In [1]:
import librosa
import pandas as pd
import pickle
import numpy as np
import os
from random import shuffle
import keras
import multiprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras import losses, models, optimizers
from keras.layers import Dense, Input, Convolution2D, BatchNormalization, Flatten, MaxPool2D, Activation, Reshape, Dropout
from keras.utils import plot_model, Sequence
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf

Using TensorFlow backend.


Class for dumping and loading pickle files of exceedingly large sizes

Reads data byte by byte and writes data byte by byte

In [0]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size


def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

Mount google drive in colab

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Load the augmented dataset

In [0]:
dataset = pickle_load('/content/gdrive/My Drive/MIDAS-CV-TASK/dataset_large.pkl')
X_train = dataset['X_train']
y_train = dataset['y_train']
X_val = dataset['X_val']
y_val = dataset['y_val']

## Construct Batch Generator for training the model

In [0]:
class MiniBatchGenerator(Sequence):

    def __init__(self, X, y) :
        self.X = X
        self.y = y
        self.batch_size = 64

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, index):
        self.input = []
        self.output = []

        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, self.X.shape[0])
        pos = 0
        for i in range(start, end):
            self.input.append(self.X[i])
            self.output.append(self.y[i])
            pos += 1

        return np.array(self.input), np.array(self.output)

# Model definition 

In [0]:
def model_definition(learning_rate):

    input = Input(shape=(28, 28, 1))

    x = (Convolution2D(32, (3, 3), padding="same", input_shape=(28, 28, 1)))(input)
    x = (BatchNormalization())(x)
    x = (Activation('relu'))(x)
    x = (MaxPool2D())(x)
    
    x = (Dropout(0.25))(x)
    
    x = (Convolution2D(64, (3, 3), padding="same"))(x)
    x = (BatchNormalization())(x)
    x = (Activation('relu'))(x)
    
    x = (MaxPool2D())(x)
    x = (Dropout(0.25))(x)
    
    x = (Convolution2D(128, (3, 3), padding="same"))(x)
    x = (BatchNormalization())(x)
    x = (Activation('relu'))(x)
    
    x = (Dropout(0.4))(x)

    x = (Flatten())(x)
    
    x = (Dense(256, activation='relu'))(x)
    x = BatchNormalization()(x)
    x = (Dropout(0.5))(x)

    x = (Dense(64, activation='relu'))(x)
    x = BatchNormalization()(x)
    x = (Dropout(0.4))(x)

    x = (Dense(16, activation='relu'))(x)
    x = (Dropout(0.3))(x)

    output = (Dense(4, activation='softmax'))(x)

    model = models.Model(inputs=input, outputs=output)

    opt = keras.optimizers.Adam(learning_rate)

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])

    plot_model(model, to_file='/content/gdrive/My Drive/MIDAS-CV-TASK/model.png', show_layer_names=True, show_shapes=True)

    print(model.summary())

    return model

Define the training and validation batch generators.

In [0]:
train_gen = MiniBatchGenerator(X_train, y_train)
val_gen = MiniBatchGenerator(X_val, y_val)

Load the model. You can also see the model architecture that I have printed.

In [8]:
model = model_definition(0.001)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)  

A learning rate function has been defined which decreases the learning rate in a geometric progreesion manner. It constantly devides the learning rate by div_step until it is greater than stop.

In [0]:
def lr_iter(start, stop, div_step) :
    while start >= stop:
        yield start
        start/= div_step

Training is started for 10 epochs for every learning rate. After every 10 epochs, the learning rate is divided by div_step which is specified.

In [10]:
print("Started training...")

EPOCHS = 15

for i in lr_iter(0.005, 0.0001, 2):
    model.optimizer.lr = i
    print ("Learning rate = "+str(i))
    model.fit_generator(generator=train_gen, epochs=EPOCHS,
                   validation_data = val_gen, use_multiprocessing=True)

Started training...
Learning rate = 0.005
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
263/263 [==============================] - 11s 43ms/step - loss: 0.8998 - acc: 0.6167 - val_loss: 0.7104 - val_acc: 0.6831
Epoch 2/15
263/263 [==============================] - 7s 28ms/step - loss: 0.7070 - acc: 0.7081 - val_loss: 0.7293 - val_acc: 0.7114
Epoch 3/15
263/263 [==============================] - 8s 29ms/step - loss: 0.6392 - acc: 0.7525 - val_loss: 0.7173 - val_acc: 0.6914
Epoch 4/15
263/263 [==============================] - 8s 30ms/step - loss: 0.5942 - acc: 0.7694 - val_loss: 0.6381 - val_acc: 0.7318
Epoch 5/15
263/263 [==============================] - 8s 30ms/step - loss: 0.5696 - acc: 0.7897 - val_loss: 0.5422 - val_acc: 0.8021
Epoch 6/15
263/263 [==============================] - 8s 31ms/step - loss: 0.5323 - acc: 0.8025 - val_loss: 0.4360 - val_acc: 0.8224
Epoch 7/15
263/263 [==============================] - 8s 30ms/step - loss: 0.5220 - acc: 0.8080 - val_loss: 0.4

Model is saved with training accuracy as 90% and validation accuracy as 89%.

In [0]:
model.save_weights('/content/gdrive/My Drive/MIDAS-CV-TASK/model-90-89.h5')